In [1]:
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torchvision
from torchvision import models,transforms

In [2]:
print("Pytorch Version:",torch.__version__)
print("TorchVision Version",torch.__version__)

Pytorch Version: 1.12.1+cu113
TorchVision Version 1.12.1+cu113


In [3]:
#学習済みモデルで検証
use_pretrained=True
net=models.vgg16(pretrained=use_pretrained)
net.eval()
print(net)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
import random
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)
torch.backends.cudnn.deterministic=True
torch.backends.cudnn.benchmark=False

In [5]:
#前処理クラス
class ImageTransform():
  def __init__(self,resize,mean,std):
    self.data_transform={
        'train':transforms.Compose([
            #スケールを0.5~1.0, アスペクト比3/4~4/3にランダム変更
            transforms.RandomResizedCrop(resize,scale=(0.5,1.0)),
            #左右反転を50%で行う
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ]),
        'val':transforms.Compose([
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean,std)
        ])
    }

  def __call__(self, img, phase='train'):
    return self.data_transform[phase](img)

In [6]:
from sklearn.datasets import fetch_lfw_people
lfw = fetch_lfw_people(data_home='./scikit_learn_data/',color=True , min_faces_per_person=100)

In [7]:
x=lfw.images
y=lfw.target

In [8]:
print(lfw.target_names)

['Colin Powell' 'Donald Rumsfeld' 'George W Bush' 'Gerhard Schroeder'
 'Tony Blair']


In [9]:
import os.path as osp
import glob
import random
import torch.utils.data as data

In [10]:
alist=["Colin_Powell","Donald_Rumsfeld","George_W_Bush","Gerhard_Schroeder","Tony_Blair"]
def make_datapath_list():
  
  path_train_list=[]
  path_test_list=[]
  for name in alist:
    print(name)
    rootpath="/content/scikit_learn_data/lfw_home/lfw_funneled/"
    target_path=osp.join(rootpath+name+'/*.jpg')
    print(target_path[25:28])
    for path in glob.glob(target_path):
      print("path:"+path)
      rando=random.random()
      if rando<=0.6:
        path_train_list.append(path)
      else:
        path_test_list.append(path)
  return path_train_list,path_test_list

In [11]:
alist[0][0:3]

'Col'

In [12]:
train_list,test_list=make_datapath_list()

Colin_Powell
a/l
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0093.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0138.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0020.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0156.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0025.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0167.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0226.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0083.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0227.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0128.jpg
path:/content/scikit_learn_data/lfw_home/lfw_funneled/Colin_Powell/Colin_Powell_0069.jpg
path

In [13]:
len(test_list)

454

In [14]:
class MakeDataset(data.Dataset):
  def __init__(self,file_list,transform=None,phase='train'):
    self.file_list=file_list
    self.transform=transform
    self.phase=phase

  def __len__(self):
    return len(self.file_list)

  def __getitem__(self, index):
    img_path=self.file_list[index]
    img=Image.open(img_path)
    img_transformed=self.transform(img,self.phase)
    if img_path[49:52]==alist[0][0:3]:
      label=0
    elif img_path[49:52]==alist[1][0:3]:
      label=1
    elif img_path[49:52]==alist[2][0:3]:
      label=2
    elif img_path[49:52]==alist[3][0:3]:
      label=3
    elif img_path[49:52]==alist[4][0:3]:
      label=4

    return img_transformed,label

In [16]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset=MakeDataset(file_list=train_list,transform=ImageTransform(size,mean,std),phase="train")
test_dataset=MakeDataset(file_list=test_list,transform=ImageTransform(size,mean,std),phase="val")

In [17]:
index=0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

torch.Size([3, 224, 224])
0


In [18]:
#DataLoaderの作成
batch_size=32
train_dataloader=torch.utils.data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_dataloader=torch.utils.data.DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
dataloaders_dict={"train":train_dataloader,"val":val_dataloader}

In [19]:
batch_iterator=iter(dataloaders_dict["train"])
inputs,labels=next(batch_iterator)
print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([3, 4, 2, 2, 2, 2, 2, 4, 2, 2, 4, 2, 0, 3, 3, 4, 3, 3, 2, 2, 1, 2, 2, 3,
        1, 2, 2, 4, 1, 2, 2, 1])


In [20]:
#モデルの作成
use_pretrained=True
net=models.vgg16(pretrained=use_pretrained)
print(net)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [21]:
import torch.nn as nn
net.classifier[6]=nn.Linear(in_features=4096,out_features=5)
print(net)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [22]:
net.train()
criterion=nn.CrossEntropyLoss()

In [29]:
#転移学習の設定
params_to_update1=[]
params_to_update2=[]
params_to_update3=[]

update_param_names_1=["features"]
update_param_names_2=["classifier.0.weight","classifier.0.bias","classifier.3.weight","classifier.3.bias"]
update_param_names_3=["classifier.6.weight","classifier.6.bias"]

for name,param in net.named_parameters():
  if update_param_names_1[0] in name:
    param.requires_grad=True
    params_to_update1.append(param)
    print("params_to_update_1:",name)

  elif name in update_param_names_2:
    param.requires_grad=True
    params_to_update2.append(param)
    print("params_to_update_2:",name)

  elif name in update_param_names_3:
    param.requires_grad=True
    params_to_update3.append(param)
    print("params_to_update_3:",name)

  else:
    param.requires_grad=False
    print("勾配計算をしない:",name)


params_to_update_1: features.0.weight
params_to_update_1: features.0.bias
params_to_update_1: features.2.weight
params_to_update_1: features.2.bias
params_to_update_1: features.5.weight
params_to_update_1: features.5.bias
params_to_update_1: features.7.weight
params_to_update_1: features.7.bias
params_to_update_1: features.10.weight
params_to_update_1: features.10.bias
params_to_update_1: features.12.weight
params_to_update_1: features.12.bias
params_to_update_1: features.14.weight
params_to_update_1: features.14.bias
params_to_update_1: features.17.weight
params_to_update_1: features.17.bias
params_to_update_1: features.19.weight
params_to_update_1: features.19.bias
params_to_update_1: features.21.weight
params_to_update_1: features.21.bias
params_to_update_1: features.24.weight
params_to_update_1: features.24.bias
params_to_update_1: features.26.weight
params_to_update_1: features.26.bias
params_to_update_1: features.28.weight
params_to_update_1: features.28.bias
params_to_update_2: 

In [35]:
import torch.optim as optim
optimizer=optim.SGD([
    {'params':params_to_update1, 'lr':1e-4},
    {'params':params_to_update2, 'lr':5e-4},
    {'params':params_to_update3, 'lr':1e-3}
],momentum=0.9)

In [36]:
from tqdm import tqdm

def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):

  device=torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
  print("device:",device)
  net.to(device)
  torch.backends.cudnn.benchmark=True

  for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch+1,num_epochs))
    print("-------")

    for phase in ["train","val"]:
      if phase=="train":
        net.train()
      else:
        net.eval()

      epoch_loss=0.0
      epoch_corrects=0
      if(epoch==0)and(phase=="train"):
        continue
      
      for inputs,labels in tqdm(dataloaders_dict[phase]):
        inputs=inputs.to(device)
        labels=labels.to(device)

        optimizer.zero_grad()
        with torch.set_grad_enabled(phase=="train"):
          outputs=net(inputs)
          loss=criterion(outputs,labels)
          _,preds=torch.max(outputs,1)

          if phase=="train":
            loss.backward()
            optimizer.step()
          
          epoch_loss+=loss.item() * inputs.size(0)
          epoch_corrects+=torch.sum(preds==labels.data)

      epoch_loss=epoch_loss / len(dataloaders_dict[phase].dataset)
      epoch_acc=epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

      print("{} Loss: {:.4f} Acc: {:4f}".format(phase,epoch_loss,epoch_acc))

In [38]:
num_epochs=10
train_model(net,dataloaders_dict,criterion,optimizer,num_epochs=num_epochs)

device: cuda:0
Epoch 1/10
-------


100%|██████████| 15/15 [00:03<00:00,  4.35it/s]


val Loss: 1.4009 Acc: 0.486784
Epoch 2/10
-------


100%|██████████| 22/22 [00:13<00:00,  1.69it/s]


train Loss: 1.3378 Acc: 0.482507


...

val Loss: 0.2928 Acc: 0.903084
Epoch 10/10
-------


100%|██████████| 22/22 [00:11<00:00,  1.91it/s]


train Loss: 0.2305 Acc: 0.927114


100%|██████████| 15/15 [00:03<00:00,  4.38it/s]

val Loss: 0.2455 Acc: 0.918502


In [39]:
save_path="./weights_fine_tuning.pth"
torch.save(net.state_dict(),save_path)

In [40]:
load_path="./weights_fine_tuning.pth"
load_weights=torch.load(load_path, map_location={'cuda:0':'cpu'})
net.load_state_dict(load_weights)

<All keys matched successfully>